In [7]:
import pandas as pd

# Supongamos que 'file.xlsx' es el nombre de tu archivo de Excel
file_path = 'Data1.xlsx'

# Crea un diccionario vacío para almacenar los DataFrames
dataframes = {}

# Lee cada hoja del archivo de Excel y la convierte en un DataFrame
for sheet_name in pd.ExcelFile(file_path).sheet_names[:10]:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    dataframes[sheet_name] = df

# Ahora puedes acceder a cada DataFrame por su nombre de hoja
# Por ejemplo, para acceder al DataFrame de la primera hoja:

In [8]:

print(dataframes[pd.ExcelFile(file_path).sheet_names[0]])


          Fecha   x1   x2              Class  Ancho     H/Pix       H.N  \
0    2024-01-01   30   33     period resting      3  0.039344  0.118033   
1    2024-01-01   33   34  period rumination      1  0.039344  0.039344   
2    2024-01-01   34   35     period resting      1  0.039344  0.039344   
3    2024-01-01   35   47  period rumination     12  0.039344  0.472131   
4    2024-01-01   47   48     period resting      1  0.039344  0.039344   
...         ...  ...  ...                ...    ...       ...       ...   
8376 2024-06-30  439  547      period eating    108  0.039344  4.249180   
8377 2024-06-30  547  575  period rumination     28  0.039344  1.101639   
8378 2024-06-30  575  589     period resting     14  0.039344  0.550820   
8379 2024-06-30  589  604  period rumination     15  0.039344  0.590164   
8380 2024-06-30  604  640      period eating     36  0.039344  1.416393   

                    H                         S.H  
0     00:07:04.918000             00:07:04.9180

In [9]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    # Borra las filas con ancho 0
    df.drop(df.query('Ancho == 0').index, inplace=True)
    # Actualizamos el índice después de eliminar las filas
    df.reset_index(drop=True, inplace=True)
    #elimina columnas
    df.drop(columns=['x1', 'x2', 'Ancho', 'H/Pix', 'S.H'], inplace=True)
    # Convierte la columna 'H' en un formato timedelta
    df['H'] = pd.to_timedelta(df['H'].astype(str))


In [10]:
muestra=dataframes['4173-Karol']
muestra

,Fecha,Class,H.N,H
0,2024-01-01,period resting,0.236066,0 days 00:14:09.836000
1,2024-01-01,period rumination,0.354098,0 days 00:21:14.754000
2,2024-01-01,period eating,1.534426,0 days 01:32:03.934000
3,2024-01-01,period other,0.039344,0 days 00:02:21.639000
4,2024-01-01,period rumination,0.550820,0 days 00:33:02.951000
...,...,...,...,...
8877,2024-06-30,period rumination,0.078689,0 days 00:04:43.279000
8878,2024-06-30,period resting,0.039344,0 days 00:02:21.639000
8879,2024-06-30,period rumination,0.118033,0 days 00:07:04.918000
8880,2024-06-30,period eating,0.039344,0 days 00:02:21.639000


In [11]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    # Agregar las columnas "desde" y "hasta"
    df["Desde"] = ''
    df["Hasta"] = ''

    # Calcular las columnas "desde" y "hasta"
    for i in range(len(df)):
        if i == 0:
            df.at[i, "Desde"] = pd.to_timedelta('00:00:00')
        else:
            start = df.at[i-1, "Hasta"]
            df.at[i, "Desde"] = start

        end = df.at[i, "H"] + df.at[i, "Desde"]
        df.at[i, "Hasta"] = end

   

In [12]:
muestra=dataframes['4173-Karol']
muestra

,Fecha,Class,H.N,H,Desde,Hasta
0,2024-01-01,period resting,0.236066,0 days 00:14:09.836000,0 days 00:00:00,0 days 00:14:09.836000
1,2024-01-01,period rumination,0.354098,0 days 00:21:14.754000,0 days 00:14:09.836000,0 days 00:35:24.590000
2,2024-01-01,period eating,1.534426,0 days 01:32:03.934000,0 days 00:35:24.590000,0 days 02:07:28.524000
3,2024-01-01,period other,0.039344,0 days 00:02:21.639000,0 days 02:07:28.524000,0 days 02:09:50.163000
4,2024-01-01,period rumination,0.550820,0 days 00:33:02.951000,0 days 02:09:50.163000,0 days 02:42:53.114000
...,...,...,...,...,...,...
8877,2024-06-30,period rumination,0.078689,0 days 00:04:43.279000,154 days 23:29:18.317000,154 days 23:34:01.596000
8878,2024-06-30,period resting,0.039344,0 days 00:02:21.639000,154 days 23:34:01.596000,154 days 23:36:23.235000
8879,2024-06-30,period rumination,0.118033,0 days 00:07:04.918000,154 days 23:36:23.235000,154 days 23:43:28.153000
8880,2024-06-30,period eating,0.039344,0 days 00:02:21.639000,154 days 23:43:28.153000,154 days 23:45:49.792000


In [9]:
from datetime import datetime, timedelta

# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    
    hourly_periods = []
    # Iterar sobre cada fila del dataframe original
    for index, row in df.iterrows():
        fecha = row["Fecha"]
        periodo = row["Class"]
        duracion_num = row["H.N"]
        duracion_timedelta = row["H"]
        desde_timedelta = row["Desde"]
        hasta_timedelta = row["Hasta"]

        start_hour = int(desde_timedelta.total_seconds() // 3600)
        end_hour = int(hasta_timedelta.total_seconds() // 3600)

        # Calcular el total de segundos de duración
        total_seconds = duracion_timedelta.total_seconds()

        for hour in range(start_hour, end_hour + 1):
            # Calcular el tiempo en segundos dentro de la hora actual
            current_hour_start = timedelta(hours=hour )
            current_hour_end = timedelta(hours=(hour + 1) )

            # Calcular la intersección de tiempo dentro de la hora actual
            period_start = max(desde_timedelta, current_hour_start)
            period_end = min(hasta_timedelta, current_hour_end)
            period_seconds = (period_end - period_start).total_seconds()

            # Añadir al DataFrame el periodo correspondiente a la hora actual
            hourly_periods.append({
                "Fecha": fecha,
                "Hora": hour % 24,
                "Periodo": periodo,
                'H.N': duracion_num, 
                "Tiempo": timedelta(seconds=period_seconds),
                "Desde": period_start,
                "Hasta": period_end ,
            })
            
    # Crear un DataFrame a partir de la lista de diccionarios
    hourly_periods_df = pd.DataFrame( hourly_periods)
    hourly_periods_df.drop(columns=['H.N','Desde','Hasta'], inplace=True)
    # Ahora agrupa y suma por fecha, hora y period
    df_grouped = hourly_periods_df.groupby(['Fecha', 'Hora', 'Periodo'])['Tiempo'].sum().reset_index()

    # Pivotar y renombrar columnas si es necesario
    df_pivot = df_grouped.pivot_table(index=['Fecha', 'Hora'], columns='Periodo', values='Tiempo', fill_value=pd.Timedelta(0)).reset_index()
    df_pivot.columns.name = None  # Para quitar el nombre del índice de columnas

    # Define the columns to convert
    time_columns = ['period eating', 'period other', 'period resting', 'period rumination']

    # Convert each timedelta column to decimal hours
    for column in time_columns:
        df_pivot[column] = (df_pivot[column].dt.total_seconds() / 3600).round(3)

    # Mostrar el DataFrame resultante
    dataframes[sheet_name] = df_pivot

In [1]:
# Itera sobre la lista de DataFrames
for sheet_name in dataframes:
    # Accede al DataFrame correspondiente
    df = dataframes[sheet_name]
    df.to_csv('../Datos1_procesados/{sheet_name}.csv', index=False)

NameError: name 'dataframes' is not defined